# SciTeX Database Module Tutorial

This notebook demonstrates the database utilities in SciTeX for SQLite and PostgreSQL database management, inspection, and maintenance.

## 1. Setup and Imports

In [ ]:
import scitex as stx
import pandas as pd
import numpy as np
import tempfile
import os
import shutil
from datetime import datetime, timedelta
import json

# Create temporary directory for database files
temp_dir = tempfile.mkdtemp(prefix="scitex_db_demo_")
print(f"Working directory: {temp_dir}")
os.chdir(temp_dir)

## 2. SQLite Database Basics

### 2.1 Creating and Connecting to Database

In [ ]:
# Create a new SQLite database
db = stx.db.SQLite3("example.db")
print(f"Database created: {db.db_path}")

# Check connection
print(f"Connected: {db.is_connected()}")

# Database summary (empty for now)
print("\nDatabase summary:")
print(db.summary)

### 2.2 Creating Tables

In [ ]:
# Create a users table
users_schema = {
    "id": "INTEGER PRIMARY KEY AUTOINCREMENT",
    "username": "TEXT UNIQUE NOT NULL",
    "email": "TEXT NOT NULL",
    "age": "INTEGER",
    "created_at": "TIMESTAMP DEFAULT CURRENT_TIMESTAMP"
}

db.create_table("users", users_schema)
print("Created 'users' table")

# Create experiments table
experiments_schema = {
    "exp_id": "INTEGER PRIMARY KEY AUTOINCREMENT",
    "user_id": "INTEGER REFERENCES users(id)",
    "name": "TEXT NOT NULL",
    "parameters": "TEXT",  # JSON string
    "results": "TEXT",     # JSON string
    "status": "TEXT DEFAULT 'pending'",
    "start_time": "TIMESTAMP",
    "end_time": "TIMESTAMP"
}

db.create_table("experiments", experiments_schema)
print("Created 'experiments' table")

# Create measurements table with BLOB data
measurements_schema = {
    "id": "INTEGER PRIMARY KEY AUTOINCREMENT",
    "exp_id": "INTEGER REFERENCES experiments(exp_id)",
    "timestamp": "TIMESTAMP",
    "sensor_data": "BLOB",  # Binary data
    "value": "REAL",
    "unit": "TEXT"
}

db.create_table("measurements", measurements_schema)
print("Created 'measurements' table")

# Show updated summary
print("\nDatabase summary:")
print(db.summary)

## 3. CRUD Operations

### 3.1 Inserting Data

In [ ]:
# Insert single user
user_data = {
    "username": "alice",
    "email": "alice@example.com",
    "age": 28
}
user_id = db.insert("users", user_data)
print(f"Inserted user with ID: {user_id}")

# Insert multiple users at once
users_batch = [
    {"username": "bob", "email": "bob@example.com", "age": 32},
    {"username": "charlie", "email": "charlie@example.com", "age": 25},
    {"username": "diana", "email": "diana@example.com", "age": 30}
]

for user in users_batch:
    db.insert("users", user)
print(f"\nInserted {len(users_batch)} additional users")

# Insert experiments
experiments = [
    {
        "user_id": 1,
        "name": "Neural Network Training",
        "parameters": json.dumps({"lr": 0.001, "epochs": 100}),
        "status": "completed",
        "start_time": datetime.now() - timedelta(hours=2),
        "end_time": datetime.now() - timedelta(hours=1)
    },
    {
        "user_id": 2,
        "name": "Data Processing",
        "parameters": json.dumps({"batch_size": 32, "normalize": True}),
        "status": "running",
        "start_time": datetime.now() - timedelta(minutes=30)
    }
]

for exp in experiments:
    db.insert("experiments", exp)
print("\nInserted experiments")

### 3.2 Querying Data

In [ ]:
# Select all users
all_users = db.select("users")
print("All users:")
for user in all_users:
    print(f"  {user}")

# Select with conditions
young_users = db.select("users", where="age < 30")
print("\nUsers under 30:")
for user in young_users:
    print(f"  {user['username']}: age {user['age']}")

# Select specific columns
emails = db.select("users", columns=["username", "email"])
print("\nUsernames and emails:")
for row in emails:
    print(f"  {row['username']}: {row['email']}")

# Join query
query = """
    SELECT u.username, e.name as experiment, e.status
    FROM users u
    JOIN experiments e ON u.id = e.user_id
"""
results = db.execute_query(query)
print("\nUser experiments:")
for row in results:
    print(f"  {row['username']}: {row['experiment']} ({row['status']})")

### 3.3 Updating Data

In [ ]:
# Update single row
db.update("users", {"age": 29}, where="username = 'alice'")
print("Updated Alice's age")

# Update experiment status
db.update(
    "experiments",
    {
        "status": "completed",
        "end_time": datetime.now(),
        "results": json.dumps({"accuracy": 0.95, "loss": 0.12})
    },
    where="status = 'running'"
)
print("Updated running experiments to completed")

# Verify updates
updated_user = db.select("users", where="username = 'alice'")[0]
print(f"\nAlice's new age: {updated_user['age']}")

completed_exps = db.select("experiments", where="status = 'completed'")
print(f"Completed experiments: {len(completed_exps)}")

## 4. Working with BLOB Data

In [ ]:
# Generate sensor data as numpy array
sensor_data = np.random.randn(100, 3)  # 100 samples, 3 channels
print(f"Original sensor data shape: {sensor_data.shape}")

# Convert to bytes for storage
sensor_bytes = sensor_data.tobytes()

# Store in database
measurement = {
    "exp_id": 1,
    "timestamp": datetime.now(),
    "sensor_data": sensor_bytes,
    "value": sensor_data.mean(),
    "unit": "mV"
}

measurement_id = db.insert("measurements", measurement)
print(f"\nStored measurement with ID: {measurement_id}")

# Retrieve and reconstruct
retrieved = db.select("measurements", where=f"id = {measurement_id}")[0]
reconstructed = np.frombuffer(retrieved['sensor_data'], dtype=np.float64).reshape(100, 3)

print(f"\nReconstructed shape: {reconstructed.shape}")
print(f"Data integrity check: {np.array_equal(sensor_data, reconstructed)}")

# Store image data example
image_data = np.random.randint(0, 255, (64, 64, 3), dtype=np.uint8)
image_bytes = image_data.tobytes()

# You can also store metadata alongside BLOB
image_measurement = {
    "exp_id": 1,
    "timestamp": datetime.now(),
    "sensor_data": image_bytes,
    "value": float(image_data.shape[0]),  # Store dimensions
    "unit": f"image_{image_data.shape}"
}

db.insert("measurements", image_measurement)
print("\nStored image data as BLOB")

## 5. Transactions

In [ ]:
# Transaction example - all or nothing
print("=== Transaction Example ===")

# Successful transaction
with db.transaction():
    # Add new user
    new_user_id = db.insert("users", {
        "username": "eve",
        "email": "eve@example.com",
        "age": 27
    })
    
    # Add experiment for new user
    db.insert("experiments", {
        "user_id": new_user_id,
        "name": "Transaction Test",
        "parameters": json.dumps({"test": True})
    })
    
    print("Transaction committed successfully")

# Failed transaction (rollback)
try:
    with db.transaction():
        # This will succeed
        db.insert("users", {
            "username": "frank",
            "email": "frank@example.com",
            "age": 35
        })
        
        # This will fail (duplicate username)
        db.insert("users", {
            "username": "alice",  # Already exists!
            "email": "alice2@example.com",
            "age": 30
        })
except Exception as e:
    print(f"\nTransaction failed and rolled back: {e}")

# Verify rollback
frank = db.select("users", where="username = 'frank'")
print(f"Frank in database: {len(frank) > 0}")

## 6. Database Inspection

In [ ]:
# Inspect database structure and contents
print("=== Database Inspection ===")

# Quick inspection
stx.db.inspect("example.db")

# Detailed inspection of specific tables
print("\n=== Detailed Table Inspection ===")
stx.db.inspect("example.db", tables=["users", "experiments"])

# Using Inspector class for programmatic access
from scitex.db._inspect import Inspector

inspector = Inspector("example.db")

# Get all table names
tables = inspector.get_table_names()
print(f"\nTables in database: {tables}")

# Get detailed info for a table
users_info = inspector.get_table_info("users")
print("\nUsers table structure:")
for col in users_info:
    print(f"  {col['name']:15} {col['type']:20} {'NOT NULL' if col['notnull'] else 'NULL':8} "
          f"{'PRIMARY KEY' if col['pk'] else ''}")

# Get summary as DataFrame
summary_df = inspector.get_summary()
print("\nDatabase summary as DataFrame:")
print(summary_df)

## 7. Handling Duplicates

In [ ]:
# Create table with potential duplicates
db.create_table("sensor_readings", {
    "id": "INTEGER PRIMARY KEY",
    "device_id": "TEXT",
    "timestamp": "TIMESTAMP",
    "temperature": "REAL",
    "humidity": "REAL"
})

# Insert data with duplicates
readings = [
    {"device_id": "SENSOR_01", "timestamp": "2024-01-01 10:00:00", "temperature": 22.5, "humidity": 45.0},
    {"device_id": "SENSOR_01", "timestamp": "2024-01-01 10:00:00", "temperature": 22.5, "humidity": 45.0},  # Duplicate
    {"device_id": "SENSOR_02", "timestamp": "2024-01-01 10:00:00", "temperature": 23.1, "humidity": 48.0},
    {"device_id": "SENSOR_01", "timestamp": "2024-01-01 11:00:00", "temperature": 23.0, "humidity": 44.0},
    {"device_id": "SENSOR_02", "timestamp": "2024-01-01 10:00:00", "temperature": 23.1, "humidity": 48.0},  # Duplicate
]

for reading in readings:
    db.insert("sensor_readings", reading)

print(f"Inserted {len(readings)} readings (including duplicates)")

# Check for duplicates (dry run)
print("\nChecking for duplicates (dry run)...")
stx.db.delete_duplicates(
    "example.db",
    "sensor_readings",
    columns=["device_id", "timestamp", "temperature", "humidity"],
    dry_run=True
)

# Actually remove duplicates
print("\nRemoving duplicates...")
stx.db.delete_duplicates(
    "example.db",
    "sensor_readings",
    columns=["device_id", "timestamp"],  # Consider these columns for uniqueness
    dry_run=False
)

# Verify
remaining = db.select("sensor_readings")
print(f"\nRemaining readings: {len(remaining)}")
for reading in remaining:
    print(f"  Device: {reading['device_id']}, Time: {reading['timestamp']}")

## 8. Indexes and Performance

In [ ]:
# Create indexes for better query performance
print("=== Index Management ===")

# Create single column index
db.create_index("idx_users_email", "users", ["email"])
print("Created index on users.email")

# Create composite index
db.create_index("idx_exp_user_status", "experiments", ["user_id", "status"])
print("Created composite index on experiments(user_id, status)")

# Create unique index
db.create_index("idx_sensor_unique", "sensor_readings", 
                ["device_id", "timestamp"], unique=True)
print("Created unique index on sensor_readings")

# List all indexes
indexes = db.execute_query(
    "SELECT name, tbl_name FROM sqlite_master WHERE type='index'"
)
print("\nAll indexes:")
for idx in indexes:
    if not idx['name'].startswith('sqlite_'):
        print(f"  {idx['name']} on {idx['tbl_name']}")

# Demonstrate query performance (conceptual)
import time

# Insert more data for performance testing
print("\nInserting test data...")
with db.transaction():
    for i in range(100):
        db.insert("users", {
            "username": f"user_{i:04d}",
            "email": f"user_{i:04d}@example.com",
            "age": 20 + i % 40
        })

# Query with index
start = time.time()
result = db.select("users", where="email = 'user_0050@example.com'")
print(f"\nIndexed query time: {(time.time() - start)*1000:.2f}ms")

## 9. Import/Export Operations

In [ ]:
# Export to CSV
print("=== Import/Export Operations ===")

# Export users table to CSV
db.export_to_csv("users", "users_export.csv")
print("Exported users table to users_export.csv")

# Read exported CSV
df_users = pd.read_csv("users_export.csv")
print(f"\nExported {len(df_users)} users")
print(df_users.head())

# Create new table for import test
db.create_table("imported_data", {
    "id": "INTEGER PRIMARY KEY",
    "name": "TEXT",
    "value": "REAL",
    "category": "TEXT"
})

# Create sample CSV
import_data = pd.DataFrame({
    "name": ["Sample A", "Sample B", "Sample C"],
    "value": [1.23, 4.56, 7.89],
    "category": ["Type1", "Type2", "Type1"]
})
import_data.to_csv("import_test.csv", index=False)

# Import from CSV
db.import_from_csv("imported_data", "import_test.csv")
print("\nImported data from CSV")

# Verify import
imported = db.select("imported_data")
print(f"Imported {len(imported)} rows")
for row in imported:
    print(f"  {row}")

## 10. Advanced Database Operations

### 10.1 Backup and Restore

In [ ]:
# Backup database
backup_path = "example_backup.db"
db.backup(backup_path)
print(f"Database backed up to: {backup_path}")

# Verify backup
backup_db = stx.db.SQLite3(backup_path)
backup_users = backup_db.select("users")
print(f"\nBackup contains {len(backup_users)} users")
backup_db.close()

# You can also work with temporary copies
with db.temporary_copy() as temp_db:
    # Operations on temp_db don't affect original
    temp_db.insert("users", {
        "username": "temp_user",
        "email": "temp@example.com",
        "age": 99
    })
    temp_users = temp_db.select("users", where="username = 'temp_user'")
    print(f"\nTemporary database has user: {len(temp_users) > 0}")

# Verify original is unchanged
orig_temp_users = db.select("users", where="username = 'temp_user'")
print(f"Original database has temp user: {len(orig_temp_users) > 0}")

### 10.2 Database Maintenance

In [ ]:
# Database maintenance operations
print("=== Database Maintenance ===")

# Get database size before
size_before = os.path.getsize("example.db")
print(f"Database size before: {size_before:,} bytes")

# Vacuum database (reclaim space)
db.vacuum()
print("\nVacuumed database")

# Get size after
size_after = os.path.getsize("example.db")
print(f"Database size after: {size_after:,} bytes")
print(f"Space saved: {size_before - size_after:,} bytes")

# Analyze database (update statistics)
db.analyze()
print("\nAnalyzed database (updated statistics)")

# Check integrity
integrity_check = db.execute_query("PRAGMA integrity_check")
print(f"\nIntegrity check: {integrity_check[0]['integrity_check']}")

## 11. Real-World Example: Experiment Tracking System

In [ ]:
class ExperimentTracker:
    """Complete experiment tracking system using SQLite."""
    
    def __init__(self, db_path="experiments.db"):
        self.db = stx.db.SQLite3(db_path)
        self._initialize_schema()
    
    def _initialize_schema(self):
        """Create tables if they don't exist."""
        # Projects table
        self.db.create_table_if_not_exists("projects", {
            "project_id": "INTEGER PRIMARY KEY AUTOINCREMENT",
            "name": "TEXT UNIQUE NOT NULL",
            "description": "TEXT",
            "created_at": "TIMESTAMP DEFAULT CURRENT_TIMESTAMP"
        })
        
        # Runs table
        self.db.create_table_if_not_exists("runs", {
            "run_id": "TEXT PRIMARY KEY",  # Use unique ID
            "project_id": "INTEGER REFERENCES projects(project_id)",
            "name": "TEXT",
            "config": "TEXT",  # JSON
            "status": "TEXT DEFAULT 'pending'",
            "start_time": "TIMESTAMP",
            "end_time": "TIMESTAMP",
            "created_at": "TIMESTAMP DEFAULT CURRENT_TIMESTAMP"
        })
        
        # Metrics table
        self.db.create_table_if_not_exists("metrics", {
            "metric_id": "INTEGER PRIMARY KEY AUTOINCREMENT",
            "run_id": "TEXT REFERENCES runs(run_id)",
            "step": "INTEGER",
            "metric_name": "TEXT",
            "value": "REAL",
            "timestamp": "TIMESTAMP DEFAULT CURRENT_TIMESTAMP"
        })
        
        # Create indexes
        self.db.create_index("idx_runs_project", "runs", ["project_id"])
        self.db.create_index("idx_metrics_run", "metrics", ["run_id", "metric_name"])
    
    def create_project(self, name, description=""):
        """Create a new project."""
        return self.db.insert("projects", {
            "name": name,
            "description": description
        })
    
    def start_run(self, project_name, run_name, config):
        """Start a new experiment run."""
        # Get project ID
        project = self.db.select("projects", where=f"name = '{project_name}'")[0]
        
        # Generate unique run ID
        run_id = stx.repro.gen_id()
        
        # Insert run
        self.db.insert("runs", {
            "run_id": run_id,
            "project_id": project['project_id'],
            "name": run_name,
            "config": json.dumps(config),
            "status": "running",
            "start_time": datetime.now()
        })
        
        return run_id
    
    def log_metric(self, run_id, metric_name, value, step):
        """Log a metric value."""
        self.db.insert("metrics", {
            "run_id": run_id,
            "step": step,
            "metric_name": metric_name,
            "value": value
        })
    
    def end_run(self, run_id, status="completed"):
        """End an experiment run."""
        self.db.update("runs", {
            "status": status,
            "end_time": datetime.now()
        }, where=f"run_id = '{run_id}'")
    
    def get_run_metrics(self, run_id):
        """Get all metrics for a run."""
        query = """
            SELECT metric_name, step, value
            FROM metrics
            WHERE run_id = ?
            ORDER BY metric_name, step
        """
        return self.db.execute_query(query, (run_id,))
    
    def compare_runs(self, run_ids, metric_name):
        """Compare a specific metric across multiple runs."""
        placeholders = ','.join(['?' for _ in run_ids])
        query = f"""
            SELECT r.run_id, r.name, m.step, m.value
            FROM runs r
            JOIN metrics m ON r.run_id = m.run_id
            WHERE r.run_id IN ({placeholders})
            AND m.metric_name = ?
            ORDER BY r.run_id, m.step
        """
        return self.db.execute_query(query, (*run_ids, metric_name))

# Use the tracker
tracker = ExperimentTracker()

# Create project
project_id = tracker.create_project(
    "Deep Learning Research",
    "Experiments with neural architectures"
)
print(f"Created project with ID: {project_id}")

# Run experiments
run_ids = []
for lr in [0.001, 0.01, 0.1]:
    config = {
        "learning_rate": lr,
        "batch_size": 32,
        "optimizer": "adam"
    }
    
    run_id = tracker.start_run(
        "Deep Learning Research",
        f"LR_{lr}_experiment",
        config
    )
    run_ids.append(run_id)
    
    # Simulate training
    for step in range(10):
        # Log metrics
        loss = 1.0 / (1 + step) * (1 + lr)  # Simulated loss
        accuracy = 1 - loss + 0.5
        
        tracker.log_metric(run_id, "loss", loss, step)
        tracker.log_metric(run_id, "accuracy", accuracy, step)
    
    tracker.end_run(run_id)
    print(f"Completed run: {run_id[:20]}... (LR={lr})")

# Compare runs
print("\nComparing loss across runs:")
comparison = tracker.compare_runs(run_ids, "loss")
df_comparison = pd.DataFrame(comparison)
print(df_comparison.pivot(index='step', columns='name', values='value').round(3))

## 12. Cleanup

In [ ]:
# Close database connections
db.close()
tracker.db.close()

# Return to original directory and cleanup
os.chdir("..")
shutil.rmtree(temp_dir)
print(f"Cleaned up temporary directory: {temp_dir}")

## Summary and Best Practices

### Key Takeaways

1. **Easy Database Management**: Simple API for SQLite and PostgreSQL
2. **Comprehensive Features**: CRUD, transactions, BLOB support, indexing
3. **Data Integrity**: Built-in duplicate detection and removal
4. **Inspection Tools**: Analyze database structure and contents
5. **Import/Export**: Easy data exchange with CSV files

### Best Practices

1. **Use Transactions**:
   ```python
   with db.transaction():
       # Multiple operations
       # All succeed or all fail
   ```

2. **Create Indexes**:
   ```python
   db.create_index("idx_name", "table", ["column1", "column2"])
   ```

3. **Handle BLOBs Properly**:
   ```python
   # Store: numpy_array.tobytes()
   # Retrieve: np.frombuffer(blob_data, dtype=original_dtype)
   ```

4. **Regular Maintenance**:
   ```python
   db.vacuum()  # Reclaim space
   db.analyze()  # Update statistics
   ```

5. **Use Inspection Tools**:
   ```python
   stx.db.inspect("database.db")  # Quick overview
   stx.db.delete_duplicates(...)  # Clean data
   ```

In [ ]:
print("\nDatabase module tutorial completed!")
print("\nNext steps:")
print("1. Design your database schema carefully")
print("2. Use indexes for frequently queried columns")
print("3. Implement proper error handling with transactions")
print("4. Regular backups and maintenance")